<a href="https://colab.research.google.com/github/seyonechithrananda/bert-loves-chemistry/blob/master/DeepChemDev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!wget -c https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
!conda install -y -c deepchem -c rdkit -c conda-forge -c omnia deepchem-gpu=2.3.0
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

--2020-04-19 22:55:25--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85055499 (81M) [application/x-sh]
Saving to: ‘Miniconda3-latest-Linux-x86_64.sh’

Miniconda3-latest-L 100%[===================>]  81.12M   346MB/s    in 0.2s    

2020-04-19 22:55:25 (346 MB/s) - ‘Miniconda3-latest-Linux-x86_64.sh’ saved [85055499/85055499]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: - done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - asn1crypto==1.3.0=py37_0
    - ca-certificates==2020.1.1=0
    - certifi==2019.11.28=py37_0
    - cffi==1.14.0=py37h2e261b9_0
    - chardet==3.0.4=py37_1003
    - conda-package-handling==1.6.0=py37h7b6447c_0
    - 

In [0]:
import tensorflow as tf
print("tf.__version__: %s" % str(tf.__version__))
device_name = tf.test.gpu_device_name()
if not device_name:
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

tf.__version__: 2.2.0-rc3
Found GPU at: /device:GPU:0


In [0]:
!git clone https://github.com/deepchem/deepchem.git

Cloning into 'deepchem'...
remote: Enumerating objects: 514, done.
remote: Counting objects: 100% (514/514), done.
remote: Compressing objects: 100% (336/336), done.
remote: Total 30445 (delta 266), reused 349 (delta 174), pack-reused 29931
Receiving objects: 100% (30445/30445), 437.76 MiB | 14.85 MiB/s, done.
Resolving deltas: 100% (22383/22383), done.


In [0]:
%cd deepchem/
!ls

/content/deepchem
CODE_OF_CONDUCT.md  datasets  Dockerfile  LICENSE	   README.md  setup.py
contrib		    deepchem  docs	  MANIFEST.in	   scripts    website
CONTRIBUTING.md     devtools  examples	  PUBLICATIONS.md  setup.cfg


In [0]:
!python setup.py install

     |████████████████████████████████| 110 kB 11.3 MB/s 
running install
[pbr] Writing ChangeLog
[pbr] Generating ChangeLog
[pbr] ChangeLog complete (0.1s)
[pbr] Generating AUTHORS
[pbr] AUTHORS complete (0.2s)
running build
running build_py
creating build
creating build/lib
creating build/lib/deepchem
creating build/lib/deepchem/utils
creating build/lib/deepchem/utils/test
copying deepchem/utils/test/test_seq.py -> build/lib/deepchem/utils/test
copying deepchem/utils/test/test_generator_evaluator.py -> build/lib/deepchem/utils/test
copying deepchem/utils/test/__init__.py -> build/lib/deepchem/utils/test
copying deepchem/utils/test/test_rdkit_util.py -> build/lib/deepchem/utils/test
creating build/lib/deepchem/dock
creating build/lib/deepchem/dock/tests
copying deepchem/dock/tests/test_pose_scoring.py -> build/lib/deepchem/dock/tests
copying deepchem/dock/tests/test_binding_pocket.py -> build/lib/deepchem/dock/tests
copying deepchem/dock/tests/test_pose_generation.py -> build/lib/deep

In [0]:
import deepchem as dc

In [0]:
delaney_tasks, delaney_datasets, transformers = dc.molnet.load_delaney(
    featurizer='GraphConv', split='index')                       
train_dataset, valid_dataset, test_dataset = delaney_datasets 

INFO:deepchem.molnet.load_function.delaney_datasets:About to featurize Delaney dataset.


Loading raw samples now.
shard_size: 8192
About to start loading CSV from /tmp/delaney-processed.csv
Loading shard 1 of size 8192.
Featurizing sample 0
Featurizing sample 1000
TIMING: featurizing shard 0 took 4.609 s


INFO:deepchem.molnet.load_function.delaney_datasets:About to split dataset with index splitter.


TIMING: dataset construction took 4.914 s
Loading dataset from disk.
TIMING: dataset construction took 0.358 s
Loading dataset from disk.
TIMING: dataset construction took 0.148 s
Loading dataset from disk.


INFO:deepchem.molnet.load_function.delaney_datasets:About to transform data.


TIMING: dataset construction took 0.153 s
Loading dataset from disk.
TIMING: dataset construction took 0.330 s
Loading dataset from disk.
TIMING: dataset construction took 0.042 s
Loading dataset from disk.
TIMING: dataset construction took 0.040 s
Loading dataset from disk.


In [0]:
# Set up to Fit models                                                     
metric = dc.metrics.Metric(dc.metrics.pearson_r2_score, np.mean) 

n_atom_feat = 75                                                 
batch_size = 64

max_atoms_train = max([mol.get_num_atoms() for mol in train_dataset.X]) 
max_atoms_valid = max([mol.get_num_atoms() for mol in valid_dataset.X])
max_atoms_test = max([mol.get_num_atoms() for mol in test_dataset.X]) 
max_atoms = max([max_atoms_train, max_atoms_valid, max_atoms_test])   

reshard_size = 512
transformer = dc.trans.DAGTransformer(max_atoms=max_atoms)       
train_dataset.reshard(reshard_size)
train_dataset = transformer.transform(train_dataset)             
valid_dataset.reshard(reshard_size)                              
valid_dataset = transformer.transform(valid_dataset)             

TIMING: dataset construction took 0.397 s
Loading dataset from disk.
TIMING: dataset construction took 7.228 s
Loading dataset from disk.
TIMING: dataset construction took 0.064 s
Loading dataset from disk.
TIMING: dataset construction took 0.912 s
Loading dataset from disk.


In [0]:
import logging
logging.basicConfig(level=logging.INFO)

model = dc.models.DAGModel(
    len(delaney_tasks),
    max_atoms=max_atoms,
    n_atom_feat=n_atom_feat,
    batch_size=batch_size,
    learning_rate=1e-3,
    use_queue=False,
    mode='regression')

# Fit trained model
model.fit(train_dataset, nb_epoch=10)

self.dropout
None


INFO:deepchem.models.keras_model:Ending global_step 100: Average loss 1.23786
INFO:deepchem.models.keras_model:Ending global_step 150: Average loss 0.896657
INFO:deepchem.models.keras_model:TIMING: model fitting took 71.182 s


0.8966574859619141

In [0]:
print("Evaluating model")                                        
train_scores = model.evaluate(train_dataset, [metric], transformers)      
valid_scores = model.evaluate(valid_dataset, [metric], transformers)    

print("Train scores")                                            
print(train_scores)

print("Validation scores")                                       
print(valid_scores)

Evaluating model
computed_metrics: [0.31721464024384016]
computed_metrics: [0.3231992121140615]
Train scores
{'mean-pearson_r2_score': 0.31721464024384016}
Validation scores
{'mean-pearson_r2_score': 0.3231992121140615}


In [0]:
# Fit trained model
model.fit(train_dataset, nb_epoch=100)

INFO:deepchem.models.keras_model:Ending global_step 200: Average loss 0.866
INFO:deepchem.models.keras_model:Ending global_step 300: Average loss 0.820278
INFO:deepchem.models.keras_model:Ending global_step 400: Average loss 0.781647
INFO:deepchem.models.keras_model:Ending global_step 500: Average loss 0.729285
INFO:deepchem.models.keras_model:Ending global_step 600: Average loss 0.695503
INFO:deepchem.models.keras_model:Ending global_step 700: Average loss 0.671432
INFO:deepchem.models.keras_model:Ending global_step 800: Average loss 0.630609
INFO:deepchem.models.keras_model:Ending global_step 900: Average loss 0.60987
INFO:deepchem.models.keras_model:Ending global_step 1000: Average loss 0.591561
INFO:deepchem.models.keras_model:Ending global_step 1100: Average loss 0.57366
INFO:deepchem.models.keras_model:Ending global_step 1200: Average loss 0.549305
INFO:deepchem.models.keras_model:Ending global_step 1300: Average loss 0.543188
INFO:deepchem.models.keras_model:Ending global_step 1

0.49271862030029295

In [0]:
print("Evaluating model")                                        
train_scores = model.evaluate(train_dataset, [metric], transformers)      
valid_scores = model.evaluate(valid_dataset, [metric], transformers)    

print("Train scores")                                            
print(train_scores)

print("Validation scores")                                       
print(valid_scores)

Evaluating model
computed_metrics: [0.5038818875008052]
computed_metrics: [0.4880202504804988]
Train scores
{'mean-pearson_r2_score': 0.5038818875008052}
Validation scores
{'mean-pearson_r2_score': 0.4880202504804988}


In [0]:
# Fit trained model
model.fit(train_dataset, nb_epoch=500)

INFO:deepchem.models.keras_model:Ending global_step 1700: Average loss 0.499869
INFO:deepchem.models.keras_model:Ending global_step 1800: Average loss 0.485994
INFO:deepchem.models.keras_model:Ending global_step 1900: Average loss 0.486522
INFO:deepchem.models.keras_model:Ending global_step 2000: Average loss 0.469998
INFO:deepchem.models.keras_model:Ending global_step 2100: Average loss 0.467689
INFO:deepchem.models.keras_model:Ending global_step 2200: Average loss 0.468983
INFO:deepchem.models.keras_model:Ending global_step 2300: Average loss 0.452751
INFO:deepchem.models.keras_model:Ending global_step 2400: Average loss 0.450793
INFO:deepchem.models.keras_model:Ending global_step 2500: Average loss 0.451137
INFO:deepchem.models.keras_model:Ending global_step 2600: Average loss 0.441291
INFO:deepchem.models.keras_model:Ending global_step 2700: Average loss 0.438804
INFO:deepchem.models.keras_model:Ending global_step 2800: Average loss 0.438398
INFO:deepchem.models.keras_model:Ending 

0.35518482208251956

In [0]:
print("Evaluating model")                                        
train_scores = model.evaluate(train_dataset, [metric], transformers)      
valid_scores = model.evaluate(valid_dataset, [metric], transformers)    

print("Train scores")                                            
print(train_scores)

print("Validation scores")                                       
print(valid_scores)

Evaluating model
computed_metrics: [0.6213904009812273]
computed_metrics: [0.601818465447746]
Train scores
{'mean-pearson_r2_score': 0.6213904009812273}
Validation scores
{'mean-pearson_r2_score': 0.601818465447746}


In [0]:
# Fit trained model
model.fit(train_dataset, nb_epoch=500)

INFO:deepchem.models.keras_model:Ending global_step 9200: Average loss 0.3597
INFO:deepchem.models.keras_model:Ending global_step 9300: Average loss 0.35628
INFO:deepchem.models.keras_model:Ending global_step 9400: Average loss 0.360096
INFO:deepchem.models.keras_model:Ending global_step 9500: Average loss 0.355958
INFO:deepchem.models.keras_model:Ending global_step 9600: Average loss 0.353042
INFO:deepchem.models.keras_model:Ending global_step 9700: Average loss 0.358052
INFO:deepchem.models.keras_model:Ending global_step 9800: Average loss 0.354426
INFO:deepchem.models.keras_model:Ending global_step 9900: Average loss 0.352942
INFO:deepchem.models.keras_model:Ending global_step 10000: Average loss 0.356845
INFO:deepchem.models.keras_model:Ending global_step 10100: Average loss 0.353021
INFO:deepchem.models.keras_model:Ending global_step 10200: Average loss 0.352107
INFO:deepchem.models.keras_model:Ending global_step 10300: Average loss 0.354827
INFO:deepchem.models.keras_model:Ending

0.3315643310546875

In [0]:
print("Evaluating model")                                        
train_scores = model.evaluate(train_dataset, [metric], transformers)      
valid_scores = model.evaluate(valid_dataset, [metric], transformers)    

print("Train scores")                                            
print(train_scores)

print("Validation scores")                                       
print(valid_scores)

Evaluating model
computed_metrics: [0.6423596529742706]
computed_metrics: [0.6184095216778696]
Train scores
{'mean-pearson_r2_score': 0.6423596529742706}
Validation scores
{'mean-pearson_r2_score': 0.6184095216778696}


In [0]:
# Fit trained model
model.fit(train_dataset, nb_epoch=2000)

INFO:deepchem.models.keras_model:Ending global_step 16700: Average loss 0.337587
INFO:deepchem.models.keras_model:Ending global_step 16800: Average loss 0.336635
INFO:deepchem.models.keras_model:Ending global_step 16900: Average loss 0.339743
INFO:deepchem.models.keras_model:Ending global_step 17000: Average loss 0.332701
INFO:deepchem.models.keras_model:Ending global_step 17100: Average loss 0.337408
INFO:deepchem.models.keras_model:Ending global_step 17200: Average loss 0.339557
INFO:deepchem.models.keras_model:Ending global_step 17300: Average loss 0.332682
INFO:deepchem.models.keras_model:Ending global_step 17400: Average loss 0.336092
INFO:deepchem.models.keras_model:Ending global_step 17500: Average loss 0.337768
INFO:deepchem.models.keras_model:Ending global_step 17600: Average loss 0.335272
INFO:deepchem.models.keras_model:Ending global_step 17700: Average loss 0.334027
INFO:deepchem.models.keras_model:Ending global_step 17800: Average loss 0.337768
INFO:deepchem.models.keras_m

0.3090912628173828

In [0]:
print("Evaluating model")                                        
train_scores = model.evaluate(train_dataset, [metric], transformers)      
valid_scores = model.evaluate(valid_dataset, [metric], transformers)    

print("Train scores")                                            
print(train_scores)

print("Validation scores")                                       
print(valid_scores)

Evaluating model
computed_metrics: [0.6659355251890768]
computed_metrics: [0.6424493435009926]
Train scores
{'mean-pearson_r2_score': 0.6659355251890768}
Validation scores
{'mean-pearson_r2_score': 0.6424493435009926}


In [0]:
# Fit trained model
model.fit(train_dataset, nb_epoch=5000)

INFO:deepchem.models.keras_model:Ending global_step 46700: Average loss 0.315696
INFO:deepchem.models.keras_model:Ending global_step 46800: Average loss 0.313762
INFO:deepchem.models.keras_model:Ending global_step 46900: Average loss 0.316829
INFO:deepchem.models.keras_model:Ending global_step 47000: Average loss 0.312974
INFO:deepchem.models.keras_model:Ending global_step 47100: Average loss 0.313195
INFO:deepchem.models.keras_model:Ending global_step 47200: Average loss 0.314972
INFO:deepchem.models.keras_model:Ending global_step 47300: Average loss 0.315011
INFO:deepchem.models.keras_model:Ending global_step 47400: Average loss 0.31276
INFO:deepchem.models.keras_model:Ending global_step 47500: Average loss 0.315957
INFO:deepchem.models.keras_model:Ending global_step 47600: Average loss 0.313035
INFO:deepchem.models.keras_model:Ending global_step 47700: Average loss 0.313456
INFO:deepchem.models.keras_model:Ending global_step 47800: Average loss 0.31589
INFO:deepchem.models.keras_mod